# Fruits and Vegetables Image Recognition


```
# This is formatted as code
```



# Install required packages (no kagglehub needed)

In [ ]:
!pip install tensorflow numpy matplotlib scipy

: 

In [ ]:
# Using local Fruits-360 dataset
DATA_DIR = './Fruits-360 dataset/fruits-360_original-size/fruits-360-original-size'

print("Path to dataset files:", DATA_DIR)

: 

Info about this dataset :
- Train folder contains 100 images per category.
- Test : contains 10 images per category.
- validation: contiants 10 images per category.

The included food items are:

Fruits: Banana, Apple, Pear, Grapes, Orange, Kiwi, Watermelon, Pomegranate, Pineapple, Mango
Vegetables: Cucumber, Carrot, Capsicum, Onion, Potato, Lemon, Tomato, Radish, Beetroot, Cabbage, Lettuce, Spinach, Soybean, Cauliflower, Bell Pepper, Chilli Pepper, Turnip, Corn, Sweetcorn, Sweet Potato, Paprika, Jalapeño, Ginger, Garlic, Peas, Eggplant

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt


## ⚙️ Model Training & TFLite Conversion Steps

This process assumes you are using Python with TensorFlow/Keras and the MobileNetV2 architecture for Transfer Learning.

### **I. Data Preparation**

1.  **Organize Dataset:** Ensure your images are split and placed into three distinct directories: `train`, `validation`, and `test`. The subdirectories within each must be named according to their **class** (e.g., `Ripe_Apple`, `Unripe_Apple`).
2.  **Define Hyperparameters:** Set the target `IMAGE_SIZE` (e.g., $224 \times 224$), `BATCH_SIZE`, and number of `EPOCHS`.
3.  **Load Data Generators:** Use `ImageDataGenerator` to load images from the directories.
    * Apply **rescaling** (e.g., `1./255`) and **Data Augmentation** (e.g., rotation, horizontal flip) to the training set.
    * Apply only **rescaling** to the validation set.
4.  **Determine Classes:** Automatically retrieve the total number of classification classes (`NUM_CLASSES`) from the data generator.

---

### **II. Model Training (CNN)**

5.  **Load Base Model:** Load the pre-trained **MobileNetV2** model, excluding its top classification layer (`include_top=False`), and using weights pre-trained on ImageNet.
6.  **Freeze Base Model:** Set the `base_model.trainable = False` to prevent the pre-trained weights from changing during the initial training phase.
7.  **Build Custom Head:** Create a `Sequential` model that stacks:
    * The `base_model`.
    * A `GlobalAveragePooling2D` layer.
    * One or more `Dense` layers (ReLU activation recommended).
    * A final `Dense` layer with `NUM_CLASSES` neurons and `softmax` activation.
8.  **Compile Model:** Configure the training process using:
    * **Optimizer:** `Adam` (with a low learning rate, e.g., $0.0001$).
    * **Loss Function:** `categorical_crossentropy`.
    * **Metrics:** `['accuracy']`.
9.  **Fit Model:** Train the model using the training data, while monitoring performance against the validation data over the specified number of epochs.
10. **Save Keras Model:** Save the trained model in the standard Keras format (e.g., `.h5` or `SavedModel`).

---

### **III. TFLite Conversion & Deployment Prep**

11. **Instantiate Converter:** Use `tf.lite.TFLiteConverter.from_keras_model()` with the saved Keras model.
12. **Optimize:** Set `converter.optimizations = [tf.lite.Optimize.DEFAULT]` to apply **Post-Training Quantization**. This significantly reduces the model size and improves mobile performance.
13. **Convert & Save TFLite:** Execute the conversion and save the result as the optimized TFLite file (e.g., `ripeness_model.tflite`).
14. **Save Labels:** Extract the ordered list of class names from the data generator and save them to a plain text file (e.g., `ripeness_labels.txt`).
15. **Deploy Assets:** Place both the `.tflite` model file and the `.txt` labels file into the `assets/` directory of your Flutter project.

Repeat steps 1 through 15 for the **Fruit Type** and **Fruit Disease** datasets to complete your three-model architecture.

## 1. Data & Model prep


In [ ]:
# Hyper params
BATCH_SIZE = 64
IMAGE_SIZE = (180, 180)
EPOCHS = 12

TRAIN_DIR = os.path.join(DATA_DIR, 'Training')
VALID_DIR = os.path.join(DATA_DIR, 'Validation')
TEST_DIR = os.path.join(DATA_DIR, 'Test')

In [ ]:
# load the data with ImageDataGenerator to load images , resize them, and apply basic data augmentation(rotaiton, flips...) to improve the model's robustness.
# Rescale to [0, 1]
train_datagen = ImageDataGenerator(
    rescale = 1./255 ,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
# no augmentation for validaiton
valid_datagen = ImageDataGenerator(
    rescale = 1./255
)
# load the training data
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode= 'categorical'
)
validation_generator = valid_datagen.flow_from_directory(
    VALID_DIR,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode= 'categorical'
)
# the number of classes for the final layer
NUM_CLASSES = train_generator.num_classes
print(f"Total classes detected : {NUM_CLASSES}")

![img](https://encrypted-tbn3.gstatic.com/licensed-image?q=tbn:ANd9GcS8ZAQqtM-09H9jSR8hOrkmPZkc9c72vG4q97zfwxLmV5101IvOKMpveIKsUGEGooWe-VT6HqSqqps5EPS0vxdXeJ5tckxYrQwiIAtTxLSFUG_rcwE)

In [ ]:
# Load base model
# Load MobileNetV2 pre-trained on ImageNet, without the top classification layer
base_model = tf.keras.applications.MobileNetV2(
    input_shape = IMAGE_SIZE + (3,),
    include_top = False,
    weights = 'imagenet'
)
# Freeze the base model to prevent weights form being updated during the training
base_model.trainable = False

In [ ]:
# Build the custom classififer Head
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation = 'relu'),
    Dropout(0.2),# regularization to prevent overfitting
    Dense(NUM_CLASSES, activation = 'softmax') # final classification layer
])
model.summary()

In [ ]:
model.compile(
    optimizer = Adam(learning_rate = 0.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

# 2. Training

In [ ]:
history = model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data = validation_generator, 

    # 💥 CRITICAL SPEED BOOST:
    workers=os.cpu_count() # Use all available CPU cores for data loading
)

In [ ]:
# save the trained keras model for potential future use
model.save('ripness_cnn_model.h5')

# 3. Plotting results

In [ ]:
# Plot training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(NUM_EPOCHS)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# 4. Convert the Keras model to TFLite

In [ ]:
# Initialize the TFLite converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply default optimization (Post-Training Quantization) for smaller size and faster inference
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model file
tflite_model_path = 'ripeness_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved to: {tflite_model_path}")

# 5. Save the Label map
since the flutter pap needs a lsit f the class names in the correct order to interpret the model's output

In [ ]:
# Get class indices and map them to class names
labels = sorted(train_generator.class_indices.items(), key=lambda x: x[1])
class_names = [name for name, index in labels]

# Save class names to a text file
labels_file_path = 'ripeness_labels.txt'
with open(labels_file_path, 'w') as f:
    f.write('\n'.join(class_names))

print(f"Label map saved to: {labels_file_path}")
print("Final Classes:", class_names)